In [ ]:
!pip install blankly

In [ ]:
!blankly init

In [3]:
import blankly

In [11]:
def init(symbol, state: blankly.StrategyState):
    # Download price data for initialization
    state.variables['history'] = state.interface.history(symbol, to=50, return_as='list',
                                                         resolution=state.resolution)['close']
    state.variables['owns_position'] = False

In [46]:
def price_event(price, symbol, state: blankly.StrategyState):
    """ This function will give an updated price every 15 seconds from our definition below """
    state.variables['history'].append(price)
    # exit if we don't have enough populated information yet
    if len(state.variables['history']) < 14:
        return 
    bbands = blankly.indicators.bbands(state.variables['history'][-14:])
    bbands_low = bbands[0]
    bbands_high = bbands[2]
    if bbands_low > price and not state.variables['owns_position']:
      if blankly.trunc(state.interface.cash/price,2) > 0:
        state.interface.market_order(symbol, side='buy', size= blankly.trunc(state.interface.cash/price,2))
        state.variables['owns_position'] = True
    elif (bbands_low < price and state.variables['owns_position']):
      sell = blankly.trunc(state.interface.account[state.base_asset].available, 2)
      if sell > 0:
        state.interface.market_order(symbol, side = 'sell', size = sell)
        state.variables['owns_position'] = False

In [31]:
def price_baseline(price,symbol,state: blankly.StrategyState):
	buy = blankly.trunc(state.interface.cash/price, 2)
	if buy > 0:
		state.interface.market_order(symbol, side='buy', size=buy)

In [52]:
if __name__ == "__main__":
    # Authenticate Alpaca Strategy
    exchange = blankly.FTX()
    # Use our strategy helper on Alpaca
    strategy = blankly.Strategy(exchange)

    # Run the price event function every time we check for a new price - by default that is 15 seconds
    strategy.add_price_event(price_event, symbol='SOL-USD', resolution='1d', init=init)
    # Start the strategy. This will begin each of the price event ticks
    # strategy.start()
    # Or backtest using this
    results = strategy.backtest(to='1y', initial_values={'USD': 10000})
    print(results)

INFO: No portfolio name to load specified, defaulting to the first in the file: (portfolio). This is fine if there is only one portfolio in use.



Backtesting...
Progress: [##########] 100% Done...

Historical Dataframes: 
Account History: 
        SOL       USD          time  Account Value (USD)
0      0.00  10000.00  1.621119e+09          10000.00000
1      0.00  10000.00  1.621119e+09          10000.00000
2      0.00  10000.00  1.621206e+09          10000.00000
3      0.00  10000.00  1.621292e+09          10000.00000
4      0.00  10000.00  1.621379e+09          10000.00000
..      ...       ...           ...                  ...
361  278.81      0.09  1.652223e+09          14156.66775
362  278.81      0.09  1.652310e+09          12393.19450
363    0.00  13621.93  1.652396e+09          13621.93000
364    0.00  13621.93  1.652483e+09          13621.93000
365    0.00  13621.93  1.652569e+09          13621.93000

[366 rows x 4 columns]
Account Returns: 
             time     value
0    1.621119e+09       NaN
1    1.621206e+09  0.000000
2    1.621292e+09  0.000000
3    1.621379e+09  0.000000
4    1.621465e+09  0.000000
..         

INFO: Make an account to take advantage of the platform backtest viewer: https://app.blankly.finance/RETIe0J8EPSQz7wizoJX0OAFb8y1/EZkgTZMLJVaZK6kNy0mv/2b2ff92c-ee41-42b3-9afb-387de9e4f894/backtest
